<a href="https://colab.research.google.com/github/debbie105/log_Filter/blob/main/log%E8%BD%89%E6%8F%9B%E5%99%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox, scrolledtext, BooleanVar, ttk
from pathlib import Path
import gzip
import pandas as pd
import chardet
import re
import sys
import os
import importlib.util

# ---------- 工具與邏輯 ----------
def get_base_path():
    if getattr(sys, 'frozen', False): return Path(sys.executable).parent
    return Path(__file__).parent

def detect_encoding(path):
    try:
        with open(path, 'rb') as f: return chardet.detect(f.read(10240)).get('encoding') or 'utf-8'
    except: return 'utf-8'

def read_log(path):
    enc = detect_encoding(path)
    opener = gzip.open if str(path).endswith('.gz') else open
    with opener(path, 'rt', encoding=enc, errors='ignore') as f:
        for line in f: yield line.rstrip('\n')

def build_split_regex(checked, custom):
    mapping = {'space': r'\s+', 'tab': r'\t+', 'comma': ',', 'jp_comma': '、', 'semicolon': ';'}
    parts = [mapping[k] for k, v in checked.items() if v]
    if custom.strip() and custom != "自訂符號":
        parts.extend([re.escape(d.strip()) for d in custom.split(',') if d.strip()])
    return re.compile(r'(?:' + '|'.join(parts or [r'\s+']) + r')+')

def apply_filters(line, or_keys, and_keys, not_keys):
    # 排除邏輯 (NOT)
    if not_keys and any(k in line for k in not_keys): return False
    # 必含邏輯 (AND)
    if and_keys and not all(k in line for k in and_keys): return False
    # 包含邏輯 (OR)
    if or_keys and not any(k in line for k in or_keys): return False
    return True

# ---------- GUI 主介面 ----------
class App:
    def __init__(self, root):
        self.root = root
        self.root.title("Log分析工具 v8.0")
        self.root.geometry("1300x900")
        self.root.minsize(1100, 750)

        self.base_dir = get_base_path()
        self.output_dir = self.base_dir / "output"
        self.log_path = None
        self.font_main = ("微軟正黑體", 11)
        self.font_bold = ("微軟正黑體", 11, "bold")
        self.font_mono = ("Consolas", 12)
        self._setup_ui()

    def _setup_ui(self):
        btn_bg = "#eeeeee"
        main_container = tk.Frame(self.root)
        main_container.pack(fill=tk.BOTH, expand=True)

        # --- 左側控制區 ---
        left_f = tk.Frame(main_container, width=320, padx=15, pady=15, bg="#f5f5f5")
        left_f.pack(side=tk.LEFT, fill=tk.Y)
        left_f.pack_propagate(False)

        # 1. 檔案資訊與選檔 (按鈕移到這裡)
        tk.Label(left_f, text="--- 檔案資訊 ---", font=self.font_bold, bg="#f5f5f5").pack(anchor='w')
        tk.Button(left_f, text="選擇檔案", command=self.select_file, bg=btn_bg, font=self.font_main).pack(fill='x', pady=5)
        self.file_label = tk.Label(left_f, text="尚未選取", fg="gray", font=self.font_main, wraplength=280, justify='left')
        self.file_label.pack(pady=(0, 15), anchor='w')

        # 2. 分隔符號
        tk.Label(left_f, text="--- 分隔符號 ---", font=self.font_bold, bg="#f5f5f5").pack(anchor='w')
        sep_frame = tk.Frame(left_f, bg="#f5f5f5")
        sep_frame.pack(fill='x', pady=5)
        self.sep_vars = {}
        for i, (k, txt) in enumerate([('space','空白'),('tab','Tab'),('comma',','),('jp_comma','、'),('semicolon',';')]):
            var = BooleanVar(value=(k=='space'))
            var.trace_add("write", lambda *args: self.auto_preview())
            cb = tk.Checkbutton(sep_frame, text=txt, variable=var, bg="#f5f5f5", font=self.font_main)
            cb.grid(row=i//2, column=i%2, sticky='w', padx=5)
            self.sep_vars[k] = var
        self.custom_sep = tk.Entry(left_f, font=self.font_main)
        self.custom_sep.insert(0, "自訂符號"); self.custom_sep.bind("<KeyRelease>", lambda e: self.auto_preview())
        self.custom_sep.pack(fill='x', pady=(5, 15))

        # 3. 高級關鍵字篩選 (OR / AND / NOT)
        tk.Label(left_f, text="--- 關鍵字篩選 (多組用逗號) ---", font=self.font_bold, bg="#f5f5f5").pack(anchor='w')

        tk.Label(left_f, text="1. 包含其中 (OR):", font=self.font_main, bg="#f5f5f5").pack(anchor='w')
        self.or_entry = tk.Entry(left_f, font=self.font_main); self.or_entry.bind("<KeyRelease>", lambda e: self.auto_preview()); self.or_entry.pack(fill='x', pady=2)

        tk.Label(left_f, text="2. 必須包含 (AND):", font=self.font_main, bg="#f5f5f5").pack(anchor='w')
        self.and_entry = tk.Entry(left_f, font=self.font_main); self.and_entry.bind("<KeyRelease>", lambda e: self.auto_preview()); self.and_entry.pack(fill='x', pady=2)

        tk.Label(left_f, text="3. 排除字 (NOT):", font=self.font_main, bg="#f5f5f5", fg="#c0392b").pack(anchor='w')
        self.not_entry = tk.Entry(left_f, font=self.font_main); self.not_entry.bind("<KeyRelease>", lambda e: self.auto_preview()); self.not_entry.pack(fill='x', pady=(2, 15))

        # 4. 輸出設定
        tk.Label(left_f, text="--- 輸出設定 ---", font=self.font_bold, bg="#f5f5f5").pack(anchor='w')
        self.out_name = tk.Entry(left_f, font=self.font_main); self.out_name.insert(0, "analyze_result"); self.out_name.pack(fill='x', pady=5)
        fo = tk.Frame(left_f, bg="#f5f5f5"); fo.pack(anchor='w')
        self.out_opts = {k: BooleanVar(value=(k=='Excel')) for k in ['Excel','CSV','TXT']}
        for k in self.out_opts: tk.Checkbutton(fo, text=k, variable=self.out_opts[k], bg="#f5f5f5", font=self.font_main).pack(side='left', padx=5)

        # --- 置底按鈕 (1:1 等大並排) ---
        bottom_area = tk.Frame(left_f, bg="#f5f5f5")
        bottom_area.pack(side=tk.BOTTOM, fill='x', pady=10)

        tk.Button(bottom_area, text="執行存檔", command=self.run, bg="#dddddd", font=self.font_bold, height=2).pack(side=tk.LEFT, fill='x', expand=True, padx=(0,5))
        tk.Button(bottom_area, text="清除重置", command=self.clear_all, bg=btn_bg, font=self.font_main, height=2).pack(side=tk.LEFT, fill='x', expand=True, padx=(5,0))

        self.status = tk.Label(left_f, text="準備就緒", fg="blue", bg="#f5f5f5", font=self.font_main); self.status.pack(side=tk.BOTTOM, pady=5)

        # --- 右側預覽區 ---
        right_f = tk.Frame(main_container, padx=10, pady=10); right_f.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        v_pane = tk.PanedWindow(right_f, orient=tk.VERTICAL, sashrelief=tk.RAISED, sashwidth=6); v_pane.pack(fill=tk.BOTH, expand=True)

        p1 = tk.Frame(v_pane); v_pane.add(p1, height=350)
        tk.Label(p1, text="【1. 原始 Log 內容】", font=self.font_bold).pack(anchor='w')
        self.txt_orig = scrolledtext.ScrolledText(p1, font=self.font_mono, bg="#ffffff"); self.txt_orig.pack(fill=tk.BOTH, expand=True)

        p2 = tk.Frame(v_pane); v_pane.add(p2, height=450)
        tk.Label(p2, text="【2. Excel 表格模式預覽】", font=self.font_bold, fg="#27ae60").pack(anchor='w')
        style = ttk.Style(); style.configure("Treeview", font=self.font_main, rowheight=28); style.configure("Treeview.Heading", font=self.font_main)
        tree_f = tk.Frame(p2); tree_f.pack(fill=tk.BOTH, expand=True)
        self.tree = ttk.Treeview(tree_f, show="headings", style="Treeview")
        sy = ttk.Scrollbar(tree_f, orient="vertical", command=self.tree.yview); sx = ttk.Scrollbar(tree_f, orient="horizontal", command=self.tree.xview)
        self.tree.configure(yscrollcommand=sy.set, xscrollcommand=sx.set); sy.pack(side="right", fill="y"); sx.pack(side="bottom", fill="x"); self.tree.pack(fill=tk.BOTH, expand=True)

    def select_file(self):
        p = filedialog.askopenfilename()
        if p: self.log_path = Path(p); self.file_label.config(text=self.log_path.name, fg="black"); self.auto_preview()

    def get_filter_keys(self):
        def clean(e): return [k.strip() for k in e.get().split(',') if k.strip()]
        return clean(self.or_entry), clean(self.and_entry), clean(self.not_entry)

    def auto_preview(self):
        if not self.log_path: return
        self.txt_orig.delete('1.0', tk.END); self.tree.delete(*self.tree.get_children())
        regex = build_split_regex({k: v.get() for k, v in self.sep_vars.items()}, self.custom_sep.get())
        or_k, and_k, not_k = self.get_filter_keys()
        try:
            count, all_rows, max_cols = 0, [], 0
            for line in read_log(self.log_path):
                if not apply_filters(line, or_k, and_k, not_k): continue
                row_data = [x for x in regex.split(line) if x]
                all_rows.append(row_data); max_cols = max(max_cols, len(row_data))
                self.txt_orig.insert(tk.END, f"[{count+1}] {line}\n")
                count += 1
                if count >= 15: break
            self.tree["columns"] = [f"C{j}" for j in range(max_cols)]
            for j in range(max_cols): self.tree.heading(f"C{j}", text=f"欄 {j+1}"); self.tree.column(f"C{j}", width=150, anchor="w")
            for row in all_rows: self.tree.insert("", "end", values=row)
        except: pass

    def clear_all(self):
        self.log_path = None; self.file_label.config(text="尚未選取", fg="gray")
        for e in [self.or_entry, self.and_entry, self.not_entry]: e.delete(0, tk.END)
        self.custom_sep.delete(0, tk.END); self.custom_sep.insert(0, "自訂符號")
        self.txt_orig.delete('1.0', tk.END); self.tree.delete(*self.tree.get_children())
        for v in self.sep_vars.values(): v.set(False)
        self.sep_vars['space'].set(True); self.status.config(text="已清空", fg="blue")

    def run(self):
        if not self.log_path: return
        self.output_dir.mkdir(exist_ok=True)
        try:
            self.status.config(text="處理中...", fg="red"); self.root.update()
            or_k, and_k, not_k = self.get_filter_keys()
            regex = build_split_regex({k: v.get() for k, v in self.sep_vars.items()}, self.custom_sep.get())
            filtered = [l for l in read_log(self.log_path) if apply_filters(l, or_k, and_k, not_k)]
            if not filtered: messagebox.showinfo("提示", "無結果"); return
            df = pd.DataFrame([[x for x in regex.split(l) if x] for l in filtered]).apply(pd.to_numeric, errors='ignore')
            fname = self.out_name.get().strip() or "output"
            if self.out_opts['Excel'].get(): df.to_excel(self.output_dir / f"{fname}.xlsx", index=False)
            if self.out_opts['CSV'].get(): df.to_csv(self.output_dir / f"{fname}.csv", index=False, encoding="utf-8-sig")
            if self.out_opts['TXT'].get():
                with open(self.output_dir / f"{fname}.txt", "w", encoding="utf-8") as f: f.write('\n'.join(filtered))
            messagebox.showinfo("成功", f"存檔完成：{len(filtered)} 筆"); self.status.config(text="完成", fg="green")
        except Exception as e: messagebox.showerror("錯誤", str(e))

if __name__ == "__main__":
    root = tk.Tk(); App(root); root.mainloop()